# 过证书验证


## 一、前言
　　在做APP安全测试时，往往会遇到抓不到数据包的情况，导致这种情况的原因有很多种，其中证书绑定是经常遇到的问题之一。如果我们在抓包时，使用了流量转发+透明代理还无法获取APP的数据包，那么大概率使用了证书绑定，除了使用插件和frida脚本hook外，这里还有一种更为简单的办法，使用objection一个命令搞定证书绑定。

　　当然，由于证书绑定的功能是由开发者自定义的，因此并不存在一个通用的解决方案，Objection也只是对常见的App所使用的网络框架中对证书校验的代码逻辑进行了Hook修改。一旦App中的代码被混淆或者使用了未知的框架，这些App的客户端校验服务器的逻辑就需要自行分析了。



## 二、前置知识
### 2.1、证书绑定
　　ssl Pinning 这种方式不仅校验服务器证书是否是系统中的可信凭证，在通信过程中甚至连系统内置的证书都不信任，而只信任App指定的证书。一旦发现服务器证书为非指定证书即停止通信，最终导致即使将抓包工具的证书安装到系统信任凭据中也无法生效。

### 2.2、frida
　　Frida是一款轻量级的hook框架，专业点的说法就是动态插桩工具，可以插入一些代码到原生App的内存空间去动态地监视和修改其行为。通俗点讲通过该框架可以对android、ios、windows、linux等平台应用进行进程注入，从而达到劫持应用的目的，通过劫持我们可以实现各种功能。
　　该框架从Java层Hook到Native层Hook无所不能，但是持久化还是要依靠Xposed框架，另外由于其过于火爆，一些厂商也研究了反调试的方法。

### 2.3、Objection
　　Objection是基于frida的集成工具，其主要功能支持Android和iOS两大移动平台。在对Android的支持中，Objection可以快速完成诸如内存搜索、类和模块搜索、方法Hook以及打印参数、返回值、调用栈等常用功能，是一个非常方便的逆向必备工具和内存漫游神器。

#### 2.3.1、Objection过证书验证

1. 手机使用代理：一般来讲代理软件分两种，一种通过走VPN隧道的方式进行代理，有些APP会直接检测VPN代理，然后不加载。另外一种通过修改手机底层的iptables进行流量转发，从而绕过部分APP的检测，其区别在于一个软件开启后，手机上方会有一个VPN图标的提示，另一个没有。此处使用postern进行代理配置。
2. 手机使用数据线连接到电脑，开启USB调试功能，电脑端使用adb工具连接手机。使用adb shell进入root模式，启动frida。
3. 使用objection工具直接对app进行绕过，因为有些APP的证书检测是从启动时就开了，所以这里我们使用启动时附加命令。
```shell
objection -g包名 explore -s "android sslpinning disable"
```
　　可以看到其会自动对相关函数进行hook，最后就能成功抓到数据包。

## 三、总结
　　无论使用哪种方法，能获取数据包进行测试才是最终目的，不论是Xposed框架的JustTrustMe插件、frida的hook脚本还是objection的内置命令，其核心都是去查找app所调用的类，并对相关函数进行hook，最终解除证书绑定。如果上述相关办法都不管用，那么说明APP可能混淆了，需要我们自己手动查找相关函数进行hook，所以掌握其相关的基本知识对我们安全测试人员还是很有必要的。


